# Introducción a la Ingeniería de Prompt

## Instalación de paquetes
Si estás corriendo este notebook en Google Colab, corre la siguiente celda para instalar los paquetes necesarios.

In [1]:
# %pip install langchain langchain_community langchain_openai

In [2]:
# Corre esta celda solo si tienes un archivo .env configurado
from dotenv import load_dotenv
load_dotenv()

True

## Concepto

La Ingeniería de Prompts se refiere a las técnicas y metodologías empleadas en la construcción de prompts efectivos para guiar a los Modelos de Lenguaje de Gran Escala (LLMs) en la generación de resultados impactantes. Esto incluye el prompting basado en aprendizaje (In-context Learning (Brown et al., 2020; Wei et al., 2022)) y el prompting específico para tareas (zero-shot Chain-of-Thought (Kojima et al., 2022)).

## Ejemplos básicos de Prompt

In [3]:
import os
from langchain_openai import ChatOpenAI 
from langchain_core.messages import HumanMessage, SystemMessage

llm = ChatOpenAI(
    model=os.getenv("MODEL"),
    openai_api_key=os.getenv("LIA_API_KEY"),
    openai_api_base=os.getenv("LIA_API_BASE"),
    max_tokens=2000,
    temperature=0.6,
)

El formato de **chat completions** sigue un esquema de interacción entre un sistema y un humano. 

En este caso, el mensaje de entrada tiene distintos tipos pero los más comunes son:

1. **SystemMessage**: Define el contexto o las instrucciones iniciales para la conversación.
   
2. **HumanMessage**: Representa la entrada del usuario o humano.


3. **AIMessage**: Representa la salida del sistema o IA. 

In [4]:
# Clasificación de texto

messages = [
    SystemMessage(
        content="Clasifica el siguiente texto en una de estas categorías: historia, tecnología, jardinería."
    ),
    HumanMessage(
        content="Los auriculares proporcionan experiencias de audio inmersivas tanto para los amantes de la música como para los jugadores."
    ),
]

llm.invoke(messages).content

'Tecnología'

In [5]:
# Análisis de sentimientos

messages = [
    SystemMessage(
        content="Analiza el siguiente texto y determina si es positivo, negativo o neutral."
    ),
    HumanMessage(
        content="Me encanta la comida de este restaurante."
    ),
]

llm.invoke(messages).content

'El texto es positivo.'

In [6]:
# Análisis de sentimiento de varios textos (+JSON)

messages = [
    SystemMessage(
        content="Clasifica el sentimiento de los siguientes reportes de acciones como positivo, neutral o negativo. Responde en formato JSON."
    ),
    HumanMessage(
        content="""
Reporte 1: Apple: el lanzamiento del nuevo iPhone ha sido un éxito

Reporte 2: Nvidia: las ventas están siendo impulsadas por la demanda de consumidores en LLMs

Reporte 3: GX oil: la demanda de energía basada en carbono está disminuyendo gracias al aumento de la energía renovable
"""
    ),
]

llm.invoke(messages).content


'```json\n{\n  "reportes": [\n    {\n      "empresa": "Apple",\n      "sentimiento": "positivo"\n    },\n    {\n      "empresa": "Nvidia",\n      "sentimiento": "positivo"\n    },\n    {\n      "empresa": "GX oil",\n      "sentimiento": "negativo"\n    }\n  ]\n}\n```'

In [7]:
# Resumen de texto

messages = [
    SystemMessage(
        content="Escribe un resumen de 30 palabras para el siguiente texto."
    ),
    HumanMessage(
        content="""
Las PCs domésticas de los años 80 fueron pioneras icónicas de la computación personal. Computadoras como el Apple II, IBM PC y Commodore 64 revolucionaron cómo vivimos y trabajamos. Introdujeron el procesamiento de textos, los juegos y las posibilidades de la computación en el hogar. La interfaz gráfica, vista en el Apple Macintosh, cambió la forma en que interactuamos con las máquinas. Estas PCs, aunque menos potentes según los estándares actuales, encendieron una revolución digital, dando forma al paisaje tecnológico actual. Ocupan un lugar especial en la historia, simbolizando el amanecer de una nueva era en la computación que sentó las bases de la era digital moderna.
"""
    ),
]

llm.invoke(messages).content

'Las PCs de los años 80, como Apple II e IBM PC, revolucionaron la computación personal, introduciendo procesamiento de textos y juegos, y sentaron las bases de la era digital moderna.'

In [8]:
# Componer texto

messages = [
    SystemMessage(
        content="Escribe un texto sobre los relojes de buceo, mencionando los siguientes hechos:"
    ),
    HumanMessage(
        content="""
- El Rolex Oyster Perpetual fue uno de los primeros en ser resistente al agua.
- Omega produjo el Seamaster en 1948 como el primer reloj de buceo diseñado específicamente.
- El Seamaster estableció un récord mundial de inmersión de 62.5m en 1955.
- Rolex produjo el primer Submariner en 1953.
- El Submariner se hizo popular también por la asociación con James Bond hasta 1995.
- James Bond comenzó a usar el Seamaster en 1995.
"""
    ),
]

llm.invoke(messages).content
    

'Los relojes de buceo han evolucionado significativamente desde sus inicios, convirtiéndose en herramientas esenciales para los entusiastas del buceo y los amantes de la relojería. Uno de los pioneros en este campo fue el Rolex Oyster Perpetual, que se destacó por ser uno de los primeros relojes resistentes al agua, estableciendo un estándar en la industria.\n\nEn 1948, Omega lanzó el Seamaster, considerado el primer reloj de buceo diseñado específicamente para esta actividad. Este innovador modelo no solo ofrecía resistencia al agua, sino que también incorporaba características que lo hacían ideal para los buceadores. En 1955, el Seamaster logró un hito impresionante al establecer un récord mundial de inmersión, alcanzando una profundidad de 62.5 metros, lo que solidificó su reputación en el mundo del buceo.\n\nPoco después, en 1953, Rolex presentó el Submariner, otro ícono en el ámbito de los relojes de buceo. Este modelo no solo se convirtió en un referente por su diseño y funcional

In [9]:
# Responder preguntas (Question Answering)

messages = [
    SystemMessage(
        content="Eres un asistente que responde preguntas. Responde en menos de 30 palabras. Apóyate en el siguiente contexto: Las PCs domésticas de los años 80 fueron pioneras icónicas de la computación personal. Computadoras como el Apple II, IBM PC y Commodore 64 revolucionaron cómo vivimos y trabajamos. Introdujeron el procesamiento de textos, los juegos y las posibilidades de la computación en el hogar. La interfaz gráfica, vista en el Apple Macintosh, cambió la forma en que interactuamos con las máquinas. Estas PCs, aunque menos potentes según los estándares actuales, encendieron una revolución digital, dando forma al paisaje tecnológico actual. Ocupan un lugar especial en la historia, simbolizando el amanecer de una nueva era en la computación que sentó las bases de la era digital moderna."
    ),
    HumanMessage(
        content="""Dime qué computadoras domésticas fueron populares en los años 80"""
    ),
]

llm.invoke(messages).content

'Las computadoras populares de los años 80 incluyen el Apple II, IBM PC y Commodore 64.'

## Referencias

- Tom Brown, Benjamin Mann, Nick Ryder, Melanie Subbiah, Jared D Kaplan, Prafulla Dhariwal,
Arvind Neelakantan, Pranav Shyam, Girish Sastry, Amanda Askell, et al. Language models are
few-shot learners. Advances in neural information processing systems, 33:1877–1901, 2020.

- Takeshi Kojima, Shixiang Shane Gu, Machel Reid, Yutaka Matsuo, and Yusuke Iwasawa. Large
language models are zero-shot reasoners. Advances in neural information processing systems, 35:
22199–22213, 2022.

- Jason Wei, Xuezhi Wang, Dale Schuurmans, Maarten Bosma, Fei Xia, Ed Chi, Quoc V Le, Denny
Zhou, et al. Chain-of-thought prompting elicits reasoning in large language models. Advances in
neural information processing systems, 35:24824–24837, 2022.